# Практические задания 14

* Лекция 30 - Файлы JSON и модуль datetime
* Лекция 31 - Файлы CSV и модуль statistics


0. ***Простая запись и чтение файлов csv и json***  
Создайте три списка: Month – с числами от 1 до 12, Income и Outcome – оба с 12 случайными числами от 2000 до 10000. Запишите эти три списка как столбцы в csv файл. Затем прочитайте этот csv-файл как список словарей. Запишите этот список словарей в json-файл.


In [31]:
import random
import csv
import json

Month = [random.randint(1,12) for i in range(12)]
Income = [random.randint(2000,10000) for i in range(12)]
Outcome = [random.randint(2000,10000) for i in range(12)]

data = list(zip(Month, Income, Outcome))

with open('pr14_0.csv', 'w', newline = '') as f:
    w = csv.writer(f)
    w.writerow(["Month", "Income", "Outcome"])
    w.writerows(data)

In [33]:
data_list = []
with open('pr14_0.csv') as f:
    reader = csv.DictReader(f)
    for row in reader:
        data_list.append(row)

data_list

[{'Month': '4', 'Income': '4003', 'Outcome': '8627'},
 {'Month': '9', 'Income': '7944', 'Outcome': '4559'},
 {'Month': '3', 'Income': '9397', 'Outcome': '5548'},
 {'Month': '9', 'Income': '8317', 'Outcome': '6852'},
 {'Month': '12', 'Income': '6614', 'Outcome': '2045'},
 {'Month': '5', 'Income': '8322', 'Outcome': '7423'},
 {'Month': '2', 'Income': '2081', 'Outcome': '4925'},
 {'Month': '11', 'Income': '2904', 'Outcome': '6913'},
 {'Month': '3', 'Income': '7792', 'Outcome': '7086'},
 {'Month': '2', 'Income': '5109', 'Outcome': '3901'},
 {'Month': '5', 'Income': '8061', 'Outcome': '7120'},
 {'Month': '8', 'Income': '5703', 'Outcome': '3330'}]

In [34]:
with open('pr14_0.json','w') as f:
    json.dump(data_list,f)

1. ***Анализ оценок по кварталам***  
Реализуйте программу, которая должна:  
* Прочитать данные из JSON-файла.  
* Для каждого ученика вычислить среднюю и медианную оценку за каждый учебный квартал:  
   * 1 квартал: январь–март  
   * 2 квартал: апрель–май (лето пропускается)  
   * 3 квартал: сентябрь–декабрь  
      * Записать результаты в CSV-файл с колонками: ученик, квартал, средняя оценка, медианная оценка  
*Данные:*  
Файл grades.json содержит список записей следующего формата:


```
[{"name": "Bob", "subject": "Science", "grade": 86, "date": "06-09-2025"},
  {"name": "Diana", "subject": "Science", "grade": 85, "date": "31-01-2025"},
  {"name": "Bob", "subject": "Literature", "grade": 60, "date": "19-07-2025"},
  {"name": "Charlie", "subject": "Literature", "grade": 78, "date": "05-08-2025"},
  {"name": "Ethan", "subject": "Literature", "grade": 69, "date": "08-04-2025"},
  {"name": "Charlie", "subject": "Science", "grade": 63, "date": "24-10-2025"},
  {"name": "Ethan", "subject": "Math", "grade": 80, "date": "30-01-2025"},
  {"name": "Alice", "subject": "Physics", "grade": 90, "date": "15-09-2025"},
  {"name": "Ethan", "subject": "Science", "grade": 63, "date": "18-09-2025"},
  ...
]
```

 Пример вывода (quarterly_report.csv):

```
Subject,Name,Quarter,Average grade,Median grade
Physics,Bob,Q4,80.2,78
Physics,Diana,Q1,88.5,88.5
Physics,Ethan,Q2,69,69
```

Идея!!!!
```
data = [
    {("math", "Bob", "Q1"): [21, 54, 45, 87, 56]},
    {("physics", "Alice", "Q2"): [33, 42, 11, 98]},
    {("math", "Alice", "Q1"): [76, 89, 91, 67]},
    {("physics", "Bob", "Q2"): [45, 32, 78, 54]}
]
```
данные сможем сортировать `sorted_by_subject = sorted(data, key=lambda x: list(x.keys())[0][0])`

In [39]:
import csv
import json
from collections import defaultdict
import statistics
from datetime import datetime

with open('grades.json') as f:
    data = json.load(f)

def get_qurt(date_str): #"06-09-2025" - входные данные
    month = datetime.strptime(date_str, "%d-%m-%Y").month
    if 1<=month<=3:
        return 'Q1'
    if 4<=month<=5:
        return 'Q2'
    if 9<=month<=12:
        return 'Q3'    

grades_by_group = defaultdict(list)

for entry in data: #entry = {"name": "Bob", "subject": "Science", "grade": 86, "date": "06-09-2025"}
    quartal = get_qurt(entry['date'])
    if quartal:
        key = (entry["subject"],entry["name"],quartal)
        grades_by_group[key].append(entry["grade"])


results =  []
for (subject, name, q), grades in grades_by_group.items():
    avg_grades = mean(grades)
    med_grades = median(grades)
    results.append([subject, name, q, avg_grades, med_grades])

with open('quarterly_report.csv','w') as f:
    writer = csv.writer(f)
    writer.writerow(['Subject,Name,Quarter,Average grade,Median grade'])
    writer.writerows(results)


2. ***Анализ продаж***  
Прочитайте файл sales.csv в список словарей с ключами fruit, store, date, revenue (то есть количество нужно будет умножить на цену). У всех значений нужно будет сделать корректный тип данных. После этого сформируйте два json-файла, один с информацией о фруктах, другой - с информацией о магазинах. Для каждого фрукта нужно будет вычислить суммарные продажи, среднюю продажу, медианную продажу, стандартное отклонение продаж, дату первой продажи, дату последней продажи. Во втором файле такая же информация, но по каждому магазину.

*Данные:*  
Файл `sales.csv` содержит список записей следующего формата:  
```
Fruit,Store,Date,Quantity,Price
Lemon,Rainbow,2023-01-01,120,1.50
Mango,Farmer,2023-01-01,120,4.80
Grape,Farmer,2023-02-01,80,2.00
```
Пример вывода (store_summary.json):  
```
{"Rainbow": 
{"total_revenue": 922.5, 
"average_revenue": 153.75, 
"median_revenue": 180.0, 
"stddev_revenue": 49.84, 
"first_sale_date": "2023-01-01", 
"last_sale_date": "2023-01-07"}, 
"Farmer": 
{"total_revenue": 1580.0, 
"average_revenue": 225.71, 
"median_revenue": 180.0, 
"stddev_revenue": 155.0, 
"first_sale_date": "2023-01-01", 
"last_sale_date": "2023-01-07"},
…
}
```

Идея!!! 
```
fruit_data = {'Lemon': [{'fruit': 'Lemon', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 1, 0, 0), 'revenue': 180.0}, {'fruit': 'Lemon', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 3, 0, 0), 'revenue': 180.0}, {'fruit': 'Lemon', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 5, 0, 0), 'revenue': 187.5}, {'fruit': 'Lemon', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 7, 0, 0), 'revenue': 195.0}],...}

store_data = {'Rainbow': [{'fruit': 'Lemon', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 1, 0, 0), 'revenue': 180.0}, {'fruit': 'Apples', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 2, 0, 0), 'revenue': 96.0}, {'fruit': 'Lemon', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 3, 0, 0), 'revenue': 180.0}, {'fruit': 'Apples', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 4, 0, 0), 'revenue': 84.0}, {'fruit': 'Lemon', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 5, 0, 0), 'revenue': 187.5}, {'fruit': 'Lemon', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 7, 0, 0), 'revenue': 195.0}], 'Farmer':
```

In [2]:
import csv
import json
from collections import defaultdict
import statistics
from datetime import datetime

# Fruit,Store,Date,Quantity,Price
# Lemon,Rainbow,2023-01-01,120,1.50

# Чтение CSV и подготовка записей
records = []
with open("sales.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        fruit = row["Fruit"]
        store = row["Store"]
        date = datetime.strptime(row["Date"], "%Y-%d-%m")
        quantity = int(row["Quantity"])
        price = float(row["Price"])
        revenue = quantity * price
        records.append({
            "fruit": fruit,
            "store": store,
            "date": date,
            "revenue": revenue
        })


# Функция для расчёта статистики
# {'Rainbow': [{'fruit': 'Lemon', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 1, 0, 0), 'revenue': 180.0}, {'fruit': 'Apples', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 2, 0, 0), 'revenue': 96.0}, {'fruit': 'Lemon', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 3, 0, 0), 'revenue': 180.0}, {'fruit': 'Apples', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 4, 0, 0), 'revenue': 84.0}, {'fruit': 'Lemon', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 5, 0, 0), 'revenue': 187.5}, {'fruit': 'Lemon', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 7, 0, 0), 'revenue': 195.0}]
def summarize(grouped_data):
    summary = {}
    for key, entries in grouped_data.items():
        revenues = [entry["revenue"] for entry in entries]
        dates = [entry["date"] for entry in entries]
        summary[key] = {
            "total_revenue": round(sum(revenues), 2),
            "average_revenue": round(mean(revenues), 2),
            "median_revenue": round(median(revenues), 2),
            "stddev_revenue": round(stdev(revenues), 2),
            "first_sale_date": min(dates).strftime("%Y-%m-%d"),
            "last_sale_date": max(dates).strftime("%Y-%m-%d")
        }
    return summary

# Группировка по фруктам и магазинам
fruit_data = defaultdict(list)
store_data = defaultdict(list)

for record in records:
    fruit_data[record["fruit"]].append(record)
    store_data[record["store"]].append(record)


# Создание сводок
fruit_summary = summarize(fruit_data)
store_summary = summarize(store_data)

# Сохранение в JSON
with open("fruit_summary.json", "w") as f:
    json.dump(fruit_summary, f)

with open("store_summary.json", "w") as f:
    json.dump(store_summary, f)

print("Файлы fruit_summary.json и store_summary.json успешно созданы.")


defaultdict(<class 'list'>, {'Lemon': [{'fruit': 'Lemon', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 1, 0, 0), 'revenue': 180.0}, {'fruit': 'Lemon', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 3, 0, 0), 'revenue': 180.0}, {'fruit': 'Lemon', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 5, 0, 0), 'revenue': 187.5}, {'fruit': 'Lemon', 'store': 'Rainbow', 'date': datetime.datetime(2023, 1, 7, 0, 0), 'revenue': 195.0}], 'Mango': [{'fruit': 'Mango', 'store': 'Farmer', 'date': datetime.datetime(2023, 1, 1, 0, 0), 'revenue': 576.0}, {'fruit': 'Mango', 'store': 'Continent', 'date': datetime.datetime(2023, 1, 4, 0, 0), 'revenue': 570.0}, {'fruit': 'Mango', 'store': 'Continent', 'date': datetime.datetime(2023, 1, 5, 0, 0), 'revenue': 660.0}, {'fruit': 'Mango', 'store': 'Continent', 'date': datetime.datetime(2023, 1, 6, 0, 0), 'revenue': 570.0}], 'Grape': [{'fruit': 'Grape', 'store': 'Farmer', 'date': datetime.datetime(2023, 1, 2, 0, 0), 'revenue': 160.0}, {'fruit'